In [1]:
%load_ext autoreload
%autoreload 2

from src.data.mocap import MotionCaptureDataset
from sklearn.decomposition import PCA
from src.utils import plot_3d

In [6]:
dataset = MotionCaptureDataset("07") ## Salsa

In [7]:
import re
pattern = re.compile("_(\d+):")
classes = [int(pattern.search(s).group(1)) for s in dataset.labels]

In [8]:
x, y, z = PCA(n_components=3).fit_transform(dataset.X).T
fig = plot_3d(x, y, z, classes = classes)
fig.show()
